<a href="https://colab.research.google.com/github/KuvinaDesch/MAT421/blob/main/ModA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are so many ways to represent numbers.  We usually use decimal, or base 10, but computers use binary, or base 2.

In decimal, there are 10 possible digits (0 to 9), and there are places for the 1s place, 10s place, 100s place, etc.  In binary, there are 2 digits, 0 and 1, and the places are the 1s place, 2s place, 4s place, 8s place, etc.

Example:
752 in base 10 is 7\*100 + 5\*10 + 2\*1
The same number in binary is 1011110000.  It is 1\*512 + 0\*256 + 1\*128 + 1\*64 + 1\*32 + 1\*16 + 0\*8 + 0\*4 + 0\*2 + 0\*1.

The rules for addition and long multiplication work the same way in decimal and binary.

This method works for integers.  For non integers, we use floating point numbers.  They are useful because we don't don't need as much precision for bigger numbers..  So there are the same number of computer numbers between 1 and 2 and between 100 and 200.  We can't represent all possible numbers with finite bits.  

With 64 bits (8 bytes), we represent a floating point number by using 1 bit for the sign, 11 for the exponent, and the remaining 52 are the fraction.  The final number equals (-1)^s \* 2^(e-1023) \* (1 + f).

Given a number x, to represent it as a 64 bit float, first take the absolute value, and find the closest power of 2 without going over.  That will be 2 to the power of something.  Add 1023 to that to get e.  Then divide x by that power of 2, to get a number from 1 to 2.  subtract 1 and this is your fraction.  Although you have to convert this to binary.  Assume all the bits for this come right after a "0." .  Finally, the sign is 1 if the number's negative.

Example:  -93.68
What is this as a 64 bit float?  It's negative so the sign will be 1.  The abs val is 93.68.  The closest power of 2 without going over is 64, or 2^6.  So the exponent is 6+1023 = 1029, or in binary, 100000101.  Finally, 93.68/64 = 1.46375.  1.46375 - 1 = 0.46375.  0.46375 = 0\*1/2 + 1\*1/4 + 1\*1/8 + 1\*1/16 + 0\*1/32 + 1\*1/64 + 1\*1/128 + 0\*1/256 + 1\*1/512 + 0\*1/1024 + ....  So the fraction part will be 0111011010...

All together, we get 11000001010111011010...

Now we'll do some python.

In [8]:
# this tells us information about what we can do with floats on this system
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [21]:
# only finitely many numbers can be represented with these floats.
# This tells us the gaps between consecutive ones around various numbers
# The higher the number, the bigger the gap
import numpy as np
print(np.spacing(5))
print(np.spacing(50))
print(np.spacing(500))
print(np.spacing(5000000))
print(np.spacing(5000000000000000))

8.881784197001252e-16
7.105427357601002e-15
5.684341886080802e-14
9.313225746154785e-10
1.0


There are exceptions to how floats are stored.  Since we can never get to 0 with a power of 2, if the exponent is -1023 (so e is 0), then we actually treat it as -1022, but the (1+f) is replaced with (0+f).  This lets us get to 0 and also as close to 0 as we can.  If the exponent is 1024 (e is 2047), then we treat it as not a number (NaN), which is undefined.  Except for the case that the fraction is 0, in which case it's infinity, or negative infinity depending on the sign.

In [72]:
# adding numbers to the maximum value doesn't change it.  This is overflow
print(sys.float_info.max)
print(sys.float_info.max+1)
print(sys.float_info.max+2)

# as we get closer and closer to 0, eventually we just get 0
print()
print(3**-500)
print(3**-600)
print(3**-700)

1.7976931348623157e+308
1.7976931348623157e+308
1.7976931348623157e+308

2.750252956203138e-239
5.336385165377108e-287
0.0


Since not every number can be represented perfectly by these floats, we often have to round to the nearest one to store it.  This difference is called round off error.  When you do math with these, the round off errors can compound.

In [97]:
# this is obviously equal to 2, but it gives us 1.9999999999999998
print(1/3 + 1/3 + 1/3 + 1/3 + 1/3 + 1/3)

# it is not equal to 2.  It says false
print(1/3 + 1/3 + 1/3 + 1/3 + 1/3 + 1/3 == 2)

# the round function lets us round to some number of decimal places to fix this
print(round(1/3 + 1/3 + 1/3 + 1/3 + 1/3 + 1/3, 10) == 2)

1.9999999999999998
False
True


In [118]:
# this function adds 1/3 to 1 n times, then subtracts it n times.
def f(n):
  x = 1
  for i in range(n):
    x += 1/3
  for i in range(n):
    x -= 1/3
  return x

# With higher values of n, the error grows.
print(f(1))
print(f(10))
print(f(100))
print(f(1000))
print(f(10000))
print(f(100000))

1.0
0.9999999999999991
1.0000000000000002
1.0000000000000064
1.0000000000001166
1.0000000000019214
